#### IOU scores between SAM and PlantSeg.

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageStat
from skimage import io, util, color, transform
from scipy.optimize import linear_sum_assignment
from scipy import ndimage
import sys
import cv2
import pickle
import re
import os
import math

In [ ]:
!pip install session-info

In [ ]:
import session_info
session_info.show()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def calculate_iou(mask1, mask2):
  '''function to calculate iou scores between a pair of masks'''
  intersection = np.logical_and(mask1, mask2)
  union = np.logical_or(mask1, mask2)
  iou = np.sum(intersection) / np.sum(union)
  return iou

In [ ]:
# initializing lists of areas, centers, masks for plantseg
areas_ps = []
centers_ps = []
mask_list_ps = []
# initializing lists of areas, centers, masks for SAM
areas_sam = []
centers_sam = []
mask_list_sam = []

# Assign folder path for mask pickle files for PlantSeg and SAM
folder_path_ps = '/content/drive/MyDrive/sanchari_ps_resized/'
folder_path_sam = '/content/drive/MyDrive/sanchari_sam_resized/'

# Loop through files in the PlantSeg folder and based on the image ID match the files to that of SAM.
# Calculate area of cell, center of mass for each cell and append to respective lists
for filename in os.listdir(folder_path_ps):
    if filename.endswith(".pkl"):  # Adjust file extensions as needed
        image_path = os.path.join(folder_path_ps, filename)
        image_number = re.search(r'(\d+)', os.path.splitext(filename)[0]).group(1)

    areas2 = []
    areas = []

    with open(image_path, 'rb') as file:
      masks_ps = pickle.load(file)

    image_path_sam = os.path.join(folder_path_sam, f"masks{image_number}.pkl")

    if os.path.exists(image_path_sam):
        with open(image_path_sam, 'rb') as file:
            masks_sam = pickle.load(file)
    else:
      continue

    for mask in masks_ps:
        # Label objects in the binary mask
        labeled_mask, num_features = ndimage.label(mask)

        # Calculate the area of each labeled object
        object_areas = ndimage.sum(mask, labeled_mask, range(1, num_features + 1))

        # Append the areas to the areas list
        areas2.append(np.sum(object_areas))

    for mask in masks_sam:
        # Label objects in the binary mask
        labeled_mask, num_features = ndimage.label(mask)

        # Calculate the area of each labeled object
        object_areas = ndimage.sum(mask, labeled_mask, range(1, num_features + 1))

        # Append the areas to the areas list
        areas.append(np.sum(object_areas))

    centers2 = []
    centers = []

    for i in range(len(masks_ps)):
      center = ndimage.measurements.center_of_mass(masks_ps[i])
      centers2.append(center)

    for i in range(len(masks_sam)):
      center = ndimage.measurements.center_of_mass(masks_sam[i])
      centers.append(center)

    areas_ps.append(areas2)
    centers_ps.append(centers2)
    mask_list_ps.append(masks_ps)

    areas_sam.append(areas)
    centers_sam.append(centers)
    mask_list_sam.append(masks_sam)

<ipython-input-6-64ac7e987a29>:54: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  center = ndimage.measurements.center_of_mass(masks_ps[i])
<ipython-input-6-64ac7e987a29>:58: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  center = ndimage.measurements.center_of_mass(masks_sam[i])


In [ ]:
def euclidean_distance(point1, point2):
    '''function to calculate euclidean distance between 2 points (in this case the center of mass of the 2 masks)'''
    if len(point1) != len(point2):
        raise ValueError("Both points must have the same number of dimensions.")

    squared_distance = sum((p1 - p2) ** 2 for p1, p2 in zip(point1, point2))
    distance = math.sqrt(squared_distance)
    return distance


In [ ]:
# Here the euclidean distances between the centers of SAM to that of PlantSeg are calculated for every image

distances = []
for i in range(len(mask_list_sam)):
  distance = []
  for center in centers_sam[i]:
    dist = []
    for center2 in centers_ps[i]:
      dist.append(euclidean_distance(center, center2))
    distance.append(dist)
  distances.append(distance)

In [ ]:
# For a particular cell mask detected by SAM, we extract 5 closest cell detected by PlantSeg (smallest euclidean distance)

map_list = []
for i in range(len(mask_list_sam)):
  fist_mapping = {}
  for outer_index, inner_list in enumerate(distances[i]):
      # Use list comprehension to get the indices of the 5 minimum values
      indices_of_min_values = sorted(range(len(inner_list)), key=lambda i: inner_list[i])[:5]

      # Store the result in the dictionary
      fist_mapping[outer_index] = indices_of_min_values

  map_list.append(fist_mapping)

In [ ]:
# we calculate the iou scores for that particular mask with all it's 5 closest masks extracted

iou_matrix_list = []
for i in range(len(mask_list_sam)):
  iou_matrix_sam = np.zeros((0, 5))
  # mask mapping wrt SAM results
  for key, values in map_list[i].items():
    iou_score = []
    for value in values:
      iou = calculate_iou(mask_list_sam[i][key], mask_list_ps[i][value])
      iou_score.append(iou)
    iou_matrix_sam = np.vstack((iou_matrix_sam, iou_score))

  iou_matrix_list.append(iou_matrix_sam)

In [ ]:
# Finally, we make the final mapping between each cell mask detected by SAM to that of PlantSeg based on the maximum of calculated IOU scores

fin_map_list = []
fin_iou_list = []
for i in range(len(mask_list_sam)):
  max_values = []
  max_indices = []

  final_mapping = {}
  final_iou_sam = []
  # Loop through the rows of the result_matrix
  for row in iou_matrix_list[i]:
      # Find the maximum value and its index in the current row
      max_value = np.max(row)
      max_index = np.argmax(row)
      # Append the maximum value and its index to the respective lists
      max_values.append(max_value)
      max_indices.append(max_index)
  k = 0
  for idx, max, value in zip(max_indices, max_values, map_list[i].values()):
    final_mapping[k] = value[idx]
    final_iou_sam.append(max)
    k = k+1
  fin_map_list.append(final_mapping)
  fin_iou_list.append(final_iou_sam)


After the final mapping we move to remove certain duplicate mappings and generate a list of IOU scores for each image

In [ ]:
fil_map_list = fin_map_list

for i in range(len(mask_list_sam)):

    for id, iou in enumerate(fin_iou_list[i]):
      fil_map_list[i][id] = (fil_map_list[i][id], iou)


for i in range(len(mask_list_sam)):
    # Create a dictionary to store the maximum IOU score for each value
    max_iou_dict = {}

    # Iterate through the dictionary and update max_iou_dict with the highest IOU for each value
    for key, value in fil_map_list[i].items():
        current_iou = value[1]
        id = value[0]
        if value[0] in max_iou_dict:
            if current_iou > max_iou_dict[value[0]][1]:
                max_iou_dict[value[0]] = (key, current_iou)
        else:
            max_iou_dict[value[0]] = (key, current_iou)

    # Update fil_map_list with only the key-value pairs with the highest IOU for each value
    fil_map_list[i] = {max_iou_dict[key][0]: key for key in max_iou_dict}


In [ ]:
iou_scores_filtered = []
for i in range(len(fil_map_list)):
    # Check if fil_map_list[i] is not empty before accessing keys
    if fil_map_list[i]:
        filtered_iou_scores = [fin_iou_list[i][idx_1] for idx_1 in fil_map_list[i].keys()]
        iou_scores_filtered.append(filtered_iou_scores)
    else:
        # Handle empty dictionaries (e.g., append an empty list or default value)
        iou_scores_filtered.append([])  # Example: append an empty list

Finally we store the IOU score in a excel file as shown below

In [ ]:
# Create a DataFrame from your list of lists
df = pd.DataFrame(iou_scores_filtered).T

# Specify the path to save the Excel file
excel_file_path = '/content/drive/MyDrive/results_iou_scores/list_iou_scores_ft_for_each_image_sanchari.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False, header=False)

print(f"List of lists saved to {excel_file_path}")

List of lists saved to /content/drive/MyDrive/results_iou_scores/list_iou_scores_ft_for_each_image_sanchari.xlsx


In [ ]:
for i in range(len(mask_list_sam)):
  print(np.mean(iou_scores_filtered[i]))

0.6274727854432006
0.3530071693522996
0.5403262597728651
0.09512793177663396
0.43464510112499677
0.6326919630428794
0.5575110619573622
0.624660371551969
0.6370158303433398
0.6710647806693373
0.5970480703152404
0.5642486885766884
0.6914751947598892
0.6933763771147067
0.6358524837427214
0.5924150985577838
0.6177267043491266
0.5842731824201366
0.5953870186388739


In [ ]:
# Calculate mean IOU scores for each sublist in iou_scores_filtered
mean_iou_scores = [np.mean(scores) for scores in iou_scores_filtered]

# Create a DataFrame with mean IOU scores
df_mean_iou = pd.DataFrame({'Mean_IOU': mean_iou_scores})

# Save DataFrame to an Excel file
excel_file_path = '/content/drive/MyDrive/results_iou_scores/mean_iou_scores_public_images.xlsx'
df_mean_iou.to_excel(excel_file_path, index=False)

print(f"Mean IOU scores saved to {excel_file_path}")

Mean IOU scores saved to /content/drive/MyDrive/results_iou_scores/mean_iou_scores_public_images.xlsx
